### 任务介绍
英雄联盟（League of Legends，LoL）是一个多人在线竞技游戏，由拳头游戏（Riot Games）公司出品。在游戏中，每位玩家控制一位有独特技能的英雄，红蓝两支队伍各有五位玩家进行对战，目标是摧毁对方的基地水晶。水晶有多座防御塔保护，通常需要先摧毁一些防御塔再摧毁水晶。玩家所控制的英雄起初非常弱，需要不断击杀小兵、野怪和对方英雄来获得金币、经验。经验可以提升英雄等级和技能等级，金币可以用来购买装备提升攻击、防御等属性。对战过程中一般没有己方单位在附近的地点是没有视野的，即无法看到对面单位，双方可以通过使用守卫来监视某个地点，洞察对面走向、制定战术。
本数据集来自[Kaggle](https://www.kaggle.com/bobbyscience/league-of-legends-diamond-ranked-games-10-min)，包含了9879场钻一到大师段位的单双排对局，对局双方几乎是同一水平。每条数据是前10分钟的对局情况，每支队伍有19个特征，红蓝双方共38个特征。这些特征包括英雄击杀、死亡，金钱、经验、等级情况等等。一局游戏一般会持续30至40分钟，但是实际前10分钟的局面很大程度上影响了之后胜负的走向。作为最成功的电子竞技游戏之一，对局数据、选手数据的量化与研究具有重要意义，可以启发游戏将来的发展和改进。

本任务是希望同学们依据注释的要求，对代码中空缺部分进行填写，**完成决策树模型的详细实现**，根据已有的对局前10分钟特征信息，预测最后获胜方是蓝色方还是红色方，了解执行一个**机器学习任务的大致流程**，并**提交代码和实验报告**。第一次作业也是一个机器学习小实验的例子，之后的作业可能不再提供预处理等流程代码，由同学们自己设计实验完成代码编写。

### 导入工具包
pandas是数据分析和处理常用的工具包，非常适合处理行列表格数据。numpy是数学运算工具包，支持高效的矩阵、向量运算。sklearn是机器学习常用工具包，包括了一些已经实现好的简单模型和一些常用数据处理方法、评价指标等函数。

In [1]:
from collections import Counter
import pandas as pd # 数据处理
import numpy as np # 数学运算
from sklearn.model_selection import train_test_split, cross_validate # 划分数据集函数
from sklearn.metrics import accuracy_score # 准确率函数
RANDOM_SEED = 2020 # 固定随机种子

### 读入数据
假设数据文件放在`./data/`目录下，标准的csv文件可以用pandas里的`read_csv()`函数直接读入。文件共有40列，38个特征（红蓝方各19），1个标签列（blueWins），和一个对局标号（gameId）。对局标号不是标签也不是特征，可以舍去。

In [2]:
csv_data = './data/high_diamond_ranked_10min.csv' # 数据路径
data_df = pd.read_csv(csv_data, sep=',') # 读入csv文件为pandas的DataFrame
data_df = data_df.drop(columns='gameId') # 舍去对局标号列

###  数据概览
对于一个机器学习问题，在拿到任务和数据后，首先需要观察数据的情况，比如我们可以通过`.iloc[0]`取出数据的第一行并输出。不难看出每个特征都存成了float64浮点数，该对局蓝色方开局10分钟有小优势。同时也可以发现有些特征列是重复冗余的，比如blueGoldDiff表示蓝色队金币优势，redGoldDiff表示红色方金币优势，这两个特征是完全对称的互为相反数。blueCSPerMin是蓝色方每分钟击杀小兵数，它乘10就是10分钟所有小兵击杀数blueTotalMinionsKilled。在之后的特征处理过程中可以考虑去除这些冗余特征。
另外，pandas有非常方便的`describe()`函数，可以直接通过DataFrame进行调用，可以展示每一列数据的一些统计信息，对数据分布情况有大致了解，比如blueKills蓝色方击杀英雄数在前十分钟的平均数是6.14、方差为2.93，中位数是6，百分之五十以上的对局中该特征在4-8之间，等等。

In [3]:
print(data_df.iloc[0]) # 输出第一行数据
data_df.describe() # 每列特征的简单统计信息

blueWins                            0.0
blueWardsPlaced                    28.0
blueWardsDestroyed                  2.0
blueFirstBlood                      1.0
blueKills                           9.0
blueDeaths                          6.0
blueAssists                        11.0
blueEliteMonsters                   0.0
blueDragons                         0.0
blueHeralds                         0.0
blueTowersDestroyed                 0.0
blueTotalGold                   17210.0
blueAvgLevel                        6.6
blueTotalExperience             17039.0
blueTotalMinionsKilled            195.0
blueTotalJungleMinionsKilled       36.0
blueGoldDiff                      643.0
blueExperienceDiff                 -8.0
blueCSPerMin                       19.5
blueGoldPerMin                   1721.0
redWardsPlaced                     15.0
redWardsDestroyed                   6.0
redFirstBlood                       0.0
redKills                            6.0
redDeaths                           9.0


,blueWins,blueWardsPlaced,blueWardsDestroyed,blueFirstBlood,blueKills,blueDeaths,blueAssists,blueEliteMonsters,blueDragons,blueHeralds,...,redTowersDestroyed,redTotalGold,redAvgLevel,redTotalExperience,redTotalMinionsKilled,redTotalJungleMinionsKilled,redGoldDiff,redExperienceDiff,redCSPerMin,redGoldPerMin
count,9879.000000,9879.000000,9879.000000,9879.000000,9879.000000,9879.000000,9879.000000,9879.000000,9879.000000,9879.000000,...,9879.000000,9879.000000,9879.000000,9879.000000,9879.000000,9879.000000,9879.000000,9879.000000,9879.000000,9879.000000
mean,0.499038,22.288288,2.824881,0.504808,6.183925,6.137666,6.645106,0.549954,0.361980,0.187974,...,0.043021,16489.041401,6.925316,17961.730438,217.349226,51.313088,-14.414111,33.620306,21.734923,1648.904140
std,0.500024,18.019177,2.174998,0.500002,3.011028,2.933818,4.064520,0.625527,0.480597,0.390712,...,0.216900,1490.888406,0.305311,1198.583912,21.911668,10.027885,2453.349179,1920.370438,2.191167,149.088841
min,0.000000,5.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,11212.000000,4.800000,10465.000000,107.000000,4.000000,-11467.000000,-8348.000000,10.700000,1121.200000
25%,0.000000,14.000000,1.000000,0.000000,4.000000,4.000000,4.000000,0.000000,0.000000,0.000000,...,0.000000,15427.500000,6.800000,17209.500000,203.000000,44.000000,-1596.000000,-1212.000000,20.300000,1542.750000
50%,0.000000,16.000000,3.000000,1.000000,6.000000,6.000000,6.000000,0.000000,0.000000,0.000000,...,0.000000,16378.000000,7.000000,17974.000000,218.000000,51.000000,-14.000000,28.000000,21.800000,1637.800000
75%,1.000000,20.000000,4.000000,1.000000,8.000000,8.000000,9.000000,1.000000,1.000000,0.000000,...,0.000000,17418.500000,7.200000,18764.500000,233.000000,57.000000,1585.500000,1290.500000,23.300000,1741.850000
max,1.000000,250.000000,27.000000,1.000000,22.000000,22.000000,29.000000,2.000000,1.000000,1.000000,...,2.000000,22732.000000,8.200000,22269.000000,289.000000,92.000000,10830.000000,9333.000000,28.900000,2273.200000


### 增删特征
传统的机器学习模型大部分都是基于特征的，因此特征工程是机器学习中非常重要的一步。有时构造一个好的特征比改进一个模型带来的提升更大。这里简单展示一些特征处理的例子。首先，上面提到，特征列中有些特征信息是完全冗余的，会给模型带来不必要的计算量，可以去除。其次，相比于红蓝双方击杀、助攻的绝对值，可能双方击杀英雄的差值更能体现出当前对战的局势。因此，我们可以构造红蓝双方对应特征的差值。数据文件中已有的差值是金币差GoldDiff和经验差ExperienceDiff，实际上每个对应特征都可以构造这样的差值特征。

In [4]:
drop_features = ['blueGoldDiff', 'redGoldDiff', 
                 'blueExperienceDiff', 'redExperienceDiff', 
                 'blueCSPerMin', 'redCSPerMin', 
                 'blueGoldPerMin', 'redGoldPerMin'] # 需要舍去的特征列
df = data_df.drop(columns=drop_features) # 舍去特征列
info_names = [c[3:] for c in df.columns if c.startswith('red')] # 取出要作差值的特征名字（除去red前缀）
for info in info_names: # 对于每个特征名字
    df['br' + info] = df['blue' + info] - df['red' + info] # 构造一个新的特征，由蓝色特征减去红色特征，前缀为br
# 其中FirstBlood为首次击杀最多有一只队伍能获得，brFirstBlood=1为蓝，0为没有产生，-1为红
df = df.drop(columns=['blueFirstBlood', 'redFirstBlood']) # 原有的FirstBlood可删除

### 特征离散化
决策树ID3算法一般是基于离散特征的，本例中存在很多连续的数值特征，例如队伍金币。直接应用该算法每个值当作一个该特征的一个取值可能造成严重的过拟合，因此需要对特征进行离散化，即将一定范围内的值映射成一个值，例如对用户年龄特征，将0-10映射到0，11-18映射到1，19-25映射到2，25-30映射到3，等等类似，然后在决策树构建时使用映射后的值计算信息增益。

In [5]:
discrete_df = df.copy() # 先复制一份数据
for c in df.columns[1:]: # 遍历每一列特征，跳过标签列
    if len(np.unique(df[c]))>10:
        discrete_df[c] = pd.qcut(df[c],([0.,0.2,0.4,0.6,0.8,1.]),labels=[0,1,2,3,4])
        continue
    else:
        continue

### 数据集准备
构建机器学习模型前要构建训练和测试的数据集。在本例中首先需要分开标签和特征，标签是不能作为模型的输入特征的，就好比作业和试卷答案不能在做题和考试前就告诉学生。测试一个模型在一个任务上的效果至少需要训练集和测试集，训练集用来训练模型的参数，好比学生做作业获得知识，测试集用来测试模型效果，好比期末考试考察学生学习情况。测试集的样本不应该出现在训练集中，否则会造成模型效果估计偏高，好比考试时出的题如果是作业题中出现过的，会造成考试分数不能准确衡量学生的学习情况，估计值偏高。划分训练集和测试集有多种方法，下面首先介绍的是随机取一部分如20%作测试集，剩下作训练集。sklearn提供了相关工具函数`train_test_split`。sklearn的输入输出一般为numpy的array矩阵，需要先将pandas的DataFrame取出为numpy的array矩阵。

In [6]:
all_y = discrete_df['blueWins'].values # 所有标签数据
feature_names = discrete_df.columns[1:] # 所有特征的名称
all_x = discrete_df[feature_names].values # 所有原始特征值，pandas的DataFrame.values取出为numpy的array矩阵

# 划分训练集和测试集
x_train, x_test, y_train, y_test = train_test_split(all_x, all_y, test_size=0.2, random_state=RANDOM_SEED)
all_y.shape, all_x.shape, x_train.shape, x_test.shape, y_train.shape, y_test.shape # 输出数据行列信息

((9879,), (9879, 43), (7903, 43), (1976, 43), (7903,), (1976,))

### 决策树模型的实现
- 根据训练数据构建一颗决策树
- 决策树会逐渐把训练集合分成越来越小的子集
- 当子集纯净后不再分裂
- 或者接受一个不完美的决策

### ID3
- 自顶向下，贪心算法
- 递归算法
- 核心循环：
  - $A$：下一步 **最佳** 决策属性
  
      Impurity(混杂度)
      
      IG(信息增益)

  - 将 $A$ 作为当前节点决策属性
  - 对 $A(vi)$ 的每个值，创建与其对应的新的子节点
  - 根据属性值将训练样本分配到各个节点
  - 如果 **训练样本被完美分类** ，则退出循环，否则继续下探分裂新的叶节点
  
      相同的输出类别 或 相同的输入类别

##### 构建决策树
1. 创建节点Node类
   - 属性
     - 数据子集x特征，y标签
     - 特征索引
     - 类别
     - 子节点(多叉数)字典存储
2. 创建决策树DecisionTree类
   - 属性
     - 根节点root
     - 最大深度
     - 最小分裂样本数
   - 方法
     - fit
       - 输入x_train(训练集特征样本)、y_train(训练集标签样本)，输出决策树
       - 创建决策树节点函数(递归)
       - 选取当前最优属性IG最大
       - 划分数据集splitDataSet
       - 递归创建子节点children
     - predict
       - 输入x_test(测试集特征样本),输出y_predict(分类结果)  
       - 遍历决策树获得分类结果

In [7]:
# 数据集：特征x(7903,43),标签y(7903,)
class Node:
    def __init__(self, x, y, cla, feature_index=None, children={}):
        self.x_data = x # 特征数据集 
        self.y_label = y # 标签数据集
        self.ftidx = feature_index # 特征索引，非叶节点特征(属性)，叶节点None
        self.cla = cla # 类别，叶节点[0,1],非叶节点None
        self.children=children #子节点字典存储

class DecisionTree(object):
    def __init__(self, classes, features, 
                 max_depth=10, min_samples_split=10): 
        self.classes = classes # classes表示模型分类总共有几类
        self.features = features # features是每个特征的名字，也方便查询总的共特征数
        self.max_depth = max_depth # 最大深度
        self.min_samples_split = min_samples_split # 最小样本数
        self.root = None # 定义根节点，未训练时为空
    
    def fit(self,x,y):
        assert len(self.features) == len(x[0])
        '''
        函数说明：信息熵计算函数
        Parameters:
            y - 标签数据集
        Returns:
            Ent - 信息熵
        '''
        def Entropy(y):
            p=0
            Ent=0
            C=Counter(y)
            for cls,freq in C.items():
                p=freq/len(y)
                Ent-=p*np.log2(p)
            return Ent


        '''
        函数说明：按照给定特征划分数据集
        Parameters:
            x - 数据集features
            y - 数据集labels
            feature_index - 特征索引
        Return:
            retx - 数据集feature划分字典(按照给定特征分类)
            rety - 数据集label划分字典(按照给定特征分类)
        '''
        def splitDataSet(x,y,feature_index):
            retx={}
            rety={}
            feature=x[:,feature_index]
            # cls给定特征类别
            cls=pd.Series(feature).unique()
            for i in cls:
                tempx=[]
                tempy=[]
                for j in range(len(x)):
                    if feature[j] == i:
                        tempx.append(x[j])
                        tempy.append(y[j])
                retx[i]=np.array(tempx)
                rety[i]=np.array(tempy)
            return retx,rety        
            
                
        '''
        函数说明：选择最优特征
        Parameters:
            x - 数据集features
            y - 数据集labels
            feature_order - 特征序列
        Returns:
            bestfeature - 最佳分裂特征(索引)
        '''
        def Bestfeature(x,y,feature_order):
            baseEntropy=Entropy(y)
            bestIG=0.0
            bestfeature=-1
            for i in range(len(self.features)):
                if i in feature_order:
                    continue
                spdatax,spdatay=splitDataSet(x,y,i)
                newEntropy=0.0
                for cls,lable in spdatay.items():
                    p=len(lable)/len(x)
                    newEntropy+=p*Entropy(lable)
                infoIG=baseEntropy-newEntropy
                if infoIG > bestIG:
                    bestIG=infoIG
                    bestfeature=i
            return bestfeature    
                

        '''
        函数说明：创建决策树(递归创建节点)
        Parameters:
            x - 数据集features
            y - 数据集labels
            infodepth - 当前节点深度
            max_depth - 最大深度
            min_sample - 最小分裂样本数
        Returns:
            node - 节点
        '''
        def CreateTree(x, y,infodepth,feature_order, max_depth, min_sample):
            if len(Counter(y)) == 1:
                return Node(x,y,y[0])
            if infodepth > max_depth:
                classes = list(Counter(y).keys())[0] if Counter(y)[0] > Counter(y)[1] else list(Counter(y).keys())[1]
                return Node(x,y,classes)
            if len(y) < min_sample:
                classes = list(Counter(y).keys())[0] if Counter(y)[0] > Counter(y)[1] else list(Counter(y).keys())[1]
                return Node(x,y,classes)
            
            bestf=Bestfeature(x,y,feature_order)
            if bestf == -1:
                classes = list(Counter(y).keys())[0] if Counter(y)[0] > Counter(y)[1] else list(Counter(y).keys())[1]
                return Node(x,y,classes,None)
            feature_order.append(bestf)
            spdatax,spdatay=splitDataSet(x,y,bestf)
            children={}
            for i in range(len(spdatax)):
                cls=list(spdatax.keys())[i]
                subx=list(spdatax.values())[i]
                suby=list(spdatay.values())[i]
                childnode=CreateTree(subx,suby,infodepth+1,feature_order,
                                     max_depth,min_sample)
                children[cls]=childnode
            node=Node(x,y,None,bestf,children=children)
            return node
            

        init_depth=0
        init_feature_od=[]
        self.root=CreateTree(x,y,init_depth,init_feature_od,self.max_depth,self.min_samples_split)
        return self
    
    
    def predict(self,x):
        assert len(x.shape) == 1 or len(x.shape) == 2 
        '''
        函数说明:分类预测，递归遍历决策树从根节点到叶子节点
        Parameters:
            data - 一个测试样本特征数据
            node - 决策树根节点
        Returns:
            class_result - 分类结果
        '''
        def classifier(data,node):
            class_result=0
            if node.cla == None:
                nextnode=node.children[data[node.ftidx]]
                class_result=classifier(data,nextnode)
            else:
                class_result=node.cla
            return class_result
        
        
        y_predict=[]
        for eachrow in x:
            yp=classifier(eachrow,self.root)
            y_predict.append(yp)
        return np.array(y_predict)
        
             

In [8]:
# 定义决策树模型，传入算法参数
DT = DecisionTree(classes=[0,1], features=feature_names, max_depth=4, min_samples_split=10) # 最大深度从0算起，5层
DT.fit(x_train, y_train) # 在训练集上训练
p_test = DT.predict(x_train) # 在测试集上预测，获得预测值
print(p_test) # 输出预测值
test_acc = accuracy_score(p_test, y_train) # 将测试预测值与测试集标签对比获得准确率
print('accuracy: {:.4f}'.format(test_acc)) # 输出准确率

[1 0 0 ... 0 0 0]
accuracy: 0.6471


In [9]:
# sklearn决策树实现(entropy)
from sklearn import tree
DT = tree.DecisionTreeClassifier(criterion='entropy',max_depth=5,min_samples_split=10)
DT.fit(x_train, y_train) 

p_test = DT.predict(x_test) 
print(p_test) 
test_acc = accuracy_score(p_test, y_test) 
print('accuracy: {:.4f}'.format(test_acc)) 

[0 1 0 ... 0 1 1]
accuracy: 0.7272


In [10]:
# sklearn决策树实现(gini)
from sklearn import tree
DT = tree.DecisionTreeClassifier(criterion='gini',max_depth=5,min_samples_split=10)
DT.fit(x_train, y_train) 

p_test = DT.predict(x_test) 
print(p_test) 
test_acc = accuracy_score(p_test, y_test) 
print('accuracy: {:.4f}'.format(test_acc)) 

[0 1 0 ... 0 1 1]
accuracy: 0.7257
